### 연습문제 2번
직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요 ?

직접 투표는 각 분류기의 예측(결과)을 모아서 가장 많이 선택된 클래스로 (다수결로) 정하는 방법이다.  
간접 투표는 분류기가 클래스의 확률을 예측할 수 있을 때, 개별 분류기의 예측 확률을 평균 내어 그 평균 확률이 가장 높은 클래스로 정하는 방법이다.  

### 연습문제 6번
에이다 부스트의 앙상블이 훈련 데이터에 과소적합되었다면 어떤 매개변수를 어떻게 바꾸어야 할까요 ?

에이다 부스트 자체만 고려한다면 예측기 수(n_estimators)를 늘리거나 학습률(learnig_rate)을 증가시켜 볼 수 있다.  
에이다 부스트에 사용하는 예측기도 고려한다면 예측기의 규제 하이퍼파라미터를 감소시켜 볼 수 있다.  

### 연습문제 8번
(3장에서 소개한) MNIST 데이터를 불러들여 훈련 세트, 검증 세트, 테스트 세트로 나눕니다(예를 들면 훈련에 50,000개 샘플, 검증에 10,000개 샘플, 테스트에 10,000개 샘플). 그런 다음 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기 같은 여러 종류의 분류기를 훈련시킵니다. 그리고 검증 세트에서 개개의 분류기보다 더 높은 성능을 내도록 이들을 간접 또는 직접 투표 방법을 사용해 앙상블로 연결해보세요. 앙상블을 얻고나면 테스트 세트로 확인해보세요. 개개의 분류기와 비교해서 성능이 얼마나 향상되나요 ?

_tensorflow import할때 계속 오류 떠서  train, test 나뉘어져있는 데이터를 다운받아 사용했습니다ㅠㅠ_

In [1]:
import sys, os

In [2]:
sys.path.append("./dataset")

In [3]:
import numpy as np
import pickle

In [4]:
from dataset.mnist import load_mnist

In [5]:
(X_train_data, y_train_data), (X_test_data, y_test_data) = load_mnist(flatten = True, normalize = False)

In [6]:
print(X_train_data.shape, y_train_data.shape)
print(X_test_data.shape, y_test_data.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_val, y_train, y_val=train_test_split(X_train_data, y_train_data, test_size=10000, random_state=42)

In [9]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

In [10]:
rf_clf=RandomForestClassifier(n_estimators=100, random_state=42)
et_clf= ExtraTreesClassifier(n_estimators=100, n_jobs=-1, random_state=42)
svm_clf=LinearSVC(max_iter=1000, random_state=42)

In [11]:
rf_clf.fit(X_train, y_train)
rf_pred=rf_clf.predict(X_val)
print(accuracy_score(y_val, rf_pred))

0.9683


In [12]:
et_clf.fit(X_train, y_train)
et_pred=et_clf.predict(X_val)
print(accuracy_score(y_val, et_pred))

0.9718


In [13]:
svm_clf.fit(X_train, y_train)
svm_pred=svm_clf.predict(X_val)
print(accuracy_score(y_val, svm_pred))

0.8639


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


#### 직접 투표

In [14]:
from sklearn.ensemble import VotingClassifier

In [17]:
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('et', et_clf), ('svc', svm_clf)], voting='hard')

In [18]:
for clf in (rf_clf, et_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    print(clf.__class__.__name__, accuracy_score(y_val, y_pred))

RandomForestClassifier 0.9683
ExtraTreesClassifier 0.9718


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC 0.8639


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


VotingClassifier 0.9687


linearSVC의 정확도는 86.39%로 randomforest와 exratree에 비해 정확도가 낮은 편이다. 따라서 linearSVC를 제외하고 앙상블로 연결하면 정확도가 더 높아질 것으로 기대할 수 있다. 

In [22]:
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('et', et_clf)], voting='hard')

In [23]:
for clf in (rf_clf, et_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test_data)
    print(clf.__class__.__name__, accuracy_score(y_test_data, y_pred))

RandomForestClassifier 0.9695
ExtraTreesClassifier 0.9698
VotingClassifier 0.97


앙상블의 정확도가 더 높다. 

#### 간접 투표

In [24]:
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('et', et_clf)], voting='soft')

In [25]:
for clf in (rf_clf, et_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test_data)
    print(clf.__class__.__name__, accuracy_score(y_test_data, y_pred))

RandomForestClassifier 0.9695
ExtraTreesClassifier 0.9698
VotingClassifier 0.9712


앙상블의 정확도가 더 높다.